In [1]:
import sys
import time
import numpy as np

import torch
from torch.utils.data import TensorDataset, DataLoader

from functions import EMA

device = 'cpu'

In [2]:
if device == 'cpu':
    torch.set_num_threads(1)

In [3]:
en_models_dict = {"Bernstein_128_8_128":{"modelEnergy_type":"Bernstein","en_elayers_dim":128,"pos_dim":8,"en_model_iter":[9]},
                  "Bernstein_128_64_128":{"modelEnergy_type":"Bernstein","en_elayers_dim":128,"pos_dim":64,"en_model_iter":[9]},
                  "Bottleneck_128_64_128":{"modelEnergy_type":"Bottleneck","en_elayers_dim":128,"pos_dim":64,"en_model_iter":[9]},
                  "Chebyshev_128_64_128":{"modelEnergy_type":"Chebyshev","en_elayers_dim":128,"pos_dim":64,"en_model_iter":[9]},
                  "Fast_128_64_128":{"modelEnergy_type":"Fast","en_elayers_dim":128,"pos_dim":64,"en_model_iter":[9]},
                  "Gram_128_64_128":{"modelEnergy_type":"Gram","en_elayers_dim":128,"pos_dim":64,"en_model_iter":[9]},
                  "Jacobi_128_8_128":{"modelEnergy_type":"Jacobi","en_elayers_dim":128,"pos_dim":8,"en_model_iter":[9]},
                  "Jacobi_128_64_128":{"modelEnergy_type":"Jacobi","en_elayers_dim":128,"pos_dim":64,"en_model_iter":[9]},
                  "Jacobi_256_128_256":{"modelEnergy_type":"Jacobi","en_elayers_dim":256,"pos_dim":128,"en_model_iter":[9]},
                  "Lagrange_128_8_128":{"modelEnergy_type":"Lagrange","en_elayers_dim":128,"pos_dim":8,"en_model_iter":[9]},
                  "Lagrange_128_64_128":{"modelEnergy_type":"Lagrange","en_elayers_dim":128,"pos_dim":64,"en_model_iter":[9]},
                  "ReLU_128_8_128":{"modelEnergy_type":"ReLU","en_elayers_dim":128,"pos_dim":8,"en_model_iter":[9]},
                  "ReLU_128_64_128":{"modelEnergy_type":"ReLU","en_elayers_dim":128,"pos_dim":64,"en_model_iter":[9]},
                  "Fast_256_32_256":{"modelEnergy_type":"Fast","en_elayers_dim":256,"pos_dim":32,"en_model_iter":[9]},
                  "Fast_256_8_256":{"modelEnergy_type":"Fast","en_elayers_dim":256,"pos_dim":8,"en_model_iter":[9]},
                  "Fast_128_128_128":{"modelEnergy_type":"Fast","en_elayers_dim":128,"pos_dim":128,"en_model_iter":[9]},
                  "Fast_128_32_128":{"modelEnergy_type":"Fast","en_elayers_dim":128,"pos_dim":32,"en_model_iter":[9]},
                  "Fast_16_128_16":{"modelEnergy_type":"Fast","en_elayers_dim":16,"pos_dim":128,"en_model_iter":[9]},
                  "Fast_16_32_16":{"modelEnergy_type":"Fast","en_elayers_dim":16,"pos_dim":32,"en_model_iter":[9]},
                  "Fast_16_8_16":{"modelEnergy_type":"Fast","en_elayers_dim":16,"pos_dim":8,"en_model_iter":[9]},
                  "Chebyshev_16_8_16":{"modelEnergy_type":"Chebyshev","en_elayers_dim":16,"pos_dim":8,"en_model_iter":[9]},
                  "SQuIRELS_256_128_256":{"modelEnergy_type":"SQuIRELS","en_elayers_dim":256,"pos_dim":128,"en_model_iter":[9]}
                  }
                    

conv_models_dict = {"Bottleneck_16_8_2":{"modelConv_type":"Bottleneck", "conv_elayers_dim":16, "temb_dim":8, "conv_dof":2, "conv_model_iter":[9]},
                    "Bottleneck_128_8_2":{"modelConv_type":"Bottleneck", "conv_elayers_dim":128, "temb_dim":8, "conv_dof":2, "conv_model_iter":[9]},
                    "Bottleneck_256_128_2":{"modelConv_type":"Bottleneck", "conv_elayers_dim":256, "temb_dim":128, "conv_dof":2, "conv_model_iter":[9]},
                    "BottleneckAttention_16_8_1":{"modelConv_type":"BottleneckAttention", "conv_elayers_dim":16, "temb_dim":8, "conv_dof":1, "conv_model_iter":[9]},
                    "Chebyshev_16_8_8":{"modelConv_type":"Chebyshev", "conv_elayers_dim":16, "temb_dim":8, "conv_dof":8, "conv_model_iter":[9]},
                    "Chebyshev_128_8_4":{"modelConv_type":"Chebyshev", "conv_elayers_dim":128, "temb_dim":8, "conv_dof":4, "conv_model_iter":[9]},
                    "Chebyshev_256_128_8":{"modelConv_type":"Chebyshev", "conv_elayers_dim":256, "temb_dim":128, "conv_dof":8, "conv_model_iter":[9]},
                    "Fast_16_8_4":{"modelConv_type":"Fast","conv_elayers_dim":16,"temb_dim":8,"conv_dof":4,"conv_model_iter":[9]},
                    "Fast_128_8_4":{"modelConv_type":"Fast", "conv_elayers_dim":128, "temb_dim":8, "conv_dof":4, "conv_model_iter":[9]},
                    "Fast_256_128_4":{"modelConv_type":"Fast", "conv_elayers_dim":256, "temb_dim":128, "conv_dof":4, "conv_model_iter":[9]},
                    "ReLU_16_8_4":{"modelConv_type":"ReLU","conv_elayers_dim":16,"temb_dim":8,"conv_dof":4,"conv_model_iter":[9]},
                    "ReLU_128_8_4":{"modelConv_type":"ReLU", "conv_elayers_dim":128, "temb_dim":8, "conv_dof":4, "conv_model_iter":[9]},
                    "ReLU_256_128_4":{"modelConv_type":"ReLU", "conv_elayers_dim":256, "temb_dim":128, "conv_dof":4, "conv_model_iter":[9]},
                    "Bernstein_16_8_1":{"modelConv_type":"Bernstein","conv_elayers_dim":16,"temb_dim":8,"conv_dof":1,"conv_model_iter":[9]},
                    "Gram_16_8_1":{"modelConv_type":"Gram", "conv_elayers_dim":16, "temb_dim":8, "conv_dof":1, "conv_model_iter":[9]},
                    "Jacobi_16_8_1":{"modelConv_type":"Jacobi","conv_elayers_dim":16,"temb_dim":8,"conv_dof":1,"conv_model_iter":[9]},
                    "Lagrange_16_8_1":{"modelConv_type":"Lagrange", "conv_elayers_dim":16, "temb_dim":8, "conv_dof":1, "conv_model_iter":[9]},
                    "SQuIRELS_256_128_32":{"modelConv_type":"SQuIRELS", "conv_elayers_dim":256, "temb_dim":128, "conv_dof":32, "conv_model_iter":[9]}
                    }

sel_en_models = en_models_dict.keys()#["Chebyshev_16_8_16"]#["SQuIRELS_256_128_256","Fast_16_8_16"] en_models_dict.keys()
sel_conv_models = conv_models_dict.keys()#["SQuIRELS_256_128_32"]#

en_models_list = ["Chebyshev_16_8_16","Fast_16_8_16","SQuIRELS_256_128_256","SQuIRELS_256_128_256"]
conv_models_list = ["Lagrange_16_8_1","Bottleneck_256_128_2","Chebyshev_128_8_4","SQuIRELS_256_128_32"]




In [4]:
i = 3

sel_en_model = en_models_dict[en_models_list[i]]
sel_conv_model = conv_models_dict[conv_models_list[i]]


# "SQuIRELS_256_128_256":{"modelEnergy_type":"SQuIRELS","en_elayers_dim":256,"pos_dim":128,"en_model_iter":[9]
modelEnergy_type = sel_en_model['modelEnergy_type']
en_elayers_dim = sel_en_model['en_elayers_dim']
pos_dim = sel_en_model['pos_dim']
en_model_iter = sel_en_model['en_model_iter']

# "SQuIRELS_256_128_32":{"modelConv_type":"SQuIRELS", "conv_elayers_dim":256, "temb_dim":128, "conv_dof":32, "conv_model_iter":[9]
modelConv_type = sel_conv_model['modelConv_type']
conv_elayers_dim = sel_conv_model['conv_elayers_dim']
temb_dim = sel_conv_model['temb_dim']
conv_dof = sel_conv_model['conv_dof']
conv_model_iter = sel_conv_model['conv_model_iter']

In [5]:
## ----------------------------------------------------------------------------------------------------
## Define Models
## ----------------------------------------------------------------------------------------------------

## Energy
if modelEnergy_type == "SQuIRELS":
    from score_models import SquirelsScoreNetwork as ScoreNetworkEnergy
elif modelEnergy_type == "Bernstein":
    from score_models import BernScoreKAN as ScoreNetworkEnergy
elif modelEnergy_type == "Bottleneck":
    from score_models import BottleneckScoreKAGN as ScoreNetworkEnergy
elif modelEnergy_type == "Chebyshev":
    from score_models import ChebyScoreKAN as ScoreNetworkEnergy
elif modelEnergy_type == "Fast":
    from score_models import FastScoreKAN as ScoreNetworkEnergy
elif modelEnergy_type == "Gram":
    from score_models import GramScoreKAN as ScoreNetworkEnergy
elif modelEnergy_type == "Jacobi":
    from score_models import JacobiScoreKAN as ScoreNetworkEnergy
elif modelEnergy_type == "Lagrange":
    from score_models import LagrangeScoreKAN as ScoreNetworkEnergy
elif modelEnergy_type == "ReLU":
    from score_models import ReluScoreKAN as ScoreNetworkEnergy
elif modelEnergy_type == "Wav":
    from score_models import WavScoreKAN as ScoreNetworkEnergy
else:
    sys.exit("Selected energy model does not exist.")
    
## Conv
if modelConv_type == "Bernstein":
    from score_models import BernScoreKANConv as ScoreNetworkConv
elif modelConv_type == "Bottleneck":
    from score_models import BottleneckScoreKAGNConv as ScoreNetworkConv
elif modelConv_type == "BottleneckAttention":
    from score_models import BottleneckScoreKAGNAttentionConv as ScoreNetworkConv
elif modelConv_type == "BottleneckKAGNLinear":
    from score_models import BottleneckScoreKAGNLinear as ScoreNetworkConv
elif modelConv_type == "Chebyshev":
    from score_models import ChebyScoreKANConv as ScoreNetworkConv
elif modelConv_type == "Fast":
    from score_models import FastScoreKANConv as ScoreNetworkConv
elif modelConv_type == "FastLinear":
    from score_models import FastScoreKANLinear as ScoreNetworkConv
elif modelConv_type == "FastWide":
    from score_models import FastScoreKANConvWide as ScoreNetworkConv
elif modelConv_type == "Gram":
    from score_models import GramScoreKANConv as ScoreNetworkConv
elif modelConv_type == "Jacobi":
    from score_models import JacobiScoreKANConv as ScoreNetworkConv
elif modelConv_type == "Lagrange":
    from score_models import LagrangeScoreKANConv as ScoreNetworkConv
elif modelConv_type == "ReLU":
    from score_models import ReluScoreKANConv as ScoreNetworkConv
elif modelConv_type == "ReLULinear":
    from score_models import ReluScoreKANLinear as ScoreNetworkConv
elif modelConv_type == "SQuIRELS":
    from score_models import SquirelsScoreNetworkConv as ScoreNetworkConv
elif modelConv_type == "SQuIRELSLinear":
    from score_models import SquirelsScoreNetworkLinear as ScoreNetworkConv
elif modelConv_type == "Wav":
    from score_models import WavScoreKANConv as ScoreNetworkConv
else :
    sys.exit("Selected convolution model does not exist.")

/home/marcelomdu/miniforge3/envs/sb_ref/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
abs_path='/mnt/f/UFRGS/TCC/Dados'

full_model_metrics = True

data_dir_path = abs_path + '/datasets/SB_Refinement'

## ----------------------------------------------------------------------------------------------------
## Energy
## ----------------------------------------------------------------------------------------------------
en_encoder_layers = [en_elayers_dim,en_elayers_dim]
en_decoder_layers = [en_elayers_dim,en_elayers_dim]
modelEnergy_version = f"{en_encoder_layers[0]}_{pos_dim}_{en_encoder_layers[0]}"

## ----------------------------------------------------------------------------------------------------
## Conv
## ----------------------------------------------------------------------------------------------------
conv_encoder_layers = [conv_elayers_dim,conv_elayers_dim]
modelConv_version = f"{conv_encoder_layers[0]}_{temb_dim}_{conv_dof}"

## ----------------------------------------------------------------------------------------------------
## Paths
## ----------------------------------------------------------------------------------------------------
models_energy_dir_path = f'{abs_path}/models/Energy/{modelEnergy_type}/{modelEnergy_version}/'
models_conv_dir_path = f'{abs_path}/models/Conv/{modelConv_type}/{modelConv_version}/'

full_modelEnergy_name = f"{modelEnergy_type}_{modelEnergy_version}"
full_modelConv_name = f"{modelConv_type}_{modelConv_version}"

plots_dir_path = f'{abs_path}/cpu/plots/Full_Models/{full_modelEnergy_name}_{full_modelConv_name}/'
if full_model_metrics:
    metrics_dir_path = f'{abs_path}/cpu/metrics/full_model_metrics/' #{full_modelEnergy_name}_{full_modelConv_name}/'
else:
    metrics_dir_path = f'{abs_path}/cpu/metrics/en_metrics/'

if en_model_iter == -1:
    energy_iter_list = range(1,10)
# elif len(en_model_iter) == 1:
#     energy_iter_list = en_model_iter*19
else:
    energy_iter_list = en_model_iter


if conv_model_iter == -1:
    conv_iter_list = range(1,10)
# elif len(conv_model_iter) == 1:
#     conv_iter_list = conv_model_iter*19
else:
    conv_iter_list = conv_model_iter

In [7]:
batch_size = 10000

d = [1,10,10]  # shape of object to diffuse
dy = [3]  # shape of object to diffuse
de = [1]  # shape of object to diffuse

num_steps_voxel = 20
gamma_max_voxel = 0.001
gamma_min_voxel = 0.001

n = num_steps_voxel//2
gamma_half_voxel = np.linspace(gamma_min_voxel, gamma_max_voxel, n)
gammas_voxel = np.concatenate([gamma_half_voxel, np.flip(gamma_half_voxel)])
gammas_voxel = torch.tensor(gammas_voxel).to(device)
T_voxel = torch.sum(gammas_voxel)

num_iter_voxel = num_steps_voxel

# print(gammas_voxel)

num_steps_energy = 20
gamma_max_energy = 0.001
gamma_min_energy = 0.001

n = num_steps_energy//2
gamma_half_energy = np.linspace(gamma_min_energy, gamma_max_energy, n)
gammas_energy = np.concatenate([gamma_half_energy, np.flip(gamma_half_energy)])
gammas_energy = torch.tensor(gammas_energy).to(device)
T_energy = torch.sum(gammas_energy)


num_iter_energy = num_steps_energy

In [8]:
## ----------------------------------------------------------------------------------------------------
## Create Models
## ----------------------------------------------------------------------------------------------------

## Energy
modelEnergy_f = ScoreNetworkEnergy(encoder_layers=en_encoder_layers,
                                pos_dim=pos_dim,
                                decoder_layers=en_decoder_layers,
                                n_cond = 1).to(device)

modelEnergy_b = ScoreNetworkEnergy(encoder_layers=en_encoder_layers,
                                pos_dim=pos_dim,
                                decoder_layers=en_decoder_layers,
                                n_cond = 1).to(device)

modelEnergy_name = str(modelEnergy_f.__class__)[21:-2]

## Conv
modelConv_f = ScoreNetworkConv(encoder_layers=conv_encoder_layers,
                            temb_dim=temb_dim,
                            conv_dof=conv_dof,
                            n_cond = 3).to(device)

modelConv_b = ScoreNetworkConv(encoder_layers=conv_encoder_layers,
                            temb_dim=temb_dim,
                            conv_dof=conv_dof,
                            n_cond = 3).to(device)

modelConv_name = str(modelConv_f.__class__)[21:-2]

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Load Model
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# modelEnergy_f = torch.nn.DataParallel(modelEnergy_f)
# modelEnergy_b = torch.nn.DataParallel(modelEnergy_b)

netEnergy_f = EMA(model=modelEnergy_f, decay=0.95).to(device)
netEnergy_b = EMA(model=modelEnergy_b, decay=0.95).to(device)

netsEnergy  = {'f': netEnergy_f, 'b': netEnergy_b }

netsEnergy['f'].load_state_dict(torch.load(f"{models_energy_dir_path}Iter{9}_net_f_GFlash_Energy{modelEnergy_name}_{modelEnergy_version}_.pth", map_location=device), strict=False)
netsEnergy['b'].load_state_dict(torch.load(f"{models_energy_dir_path}Iter{9}_net_b_GFlash_Energy{modelEnergy_name}_{modelEnergy_version}_.pth", map_location=device), strict=False)

netsEnergy['f'].eval()
netsEnergy['b'].eval()

#---------------------
netsEnergy['f'] = netsEnergy['f'].to(device)#.cpu()
netsEnergy['b'] = netsEnergy['b'].to(device)#.cpu()
# modelEnergy_f = torch.nn.DataParallel(netsEnergy['f'])
# modelEnergy_b = torch.nn.DataParallel(netsEnergy['b'])
#---------------------
print(next(netsEnergy['f'].parameters()).is_cuda)
print(next(netsEnergy['b'].parameters()).is_cuda)

# modelConv_f = torch.nn.DataParallel(modelConv_f)
# modelConv_b = torch.nn.DataParallel(modelConv_b)


netConv_f = EMA(model=modelConv_f, decay=0.95).to(device)
netConv_b = EMA(model=modelConv_b, decay=0.95).to(device)

netsConv  = {'f': netConv_f, 'b': netConv_b }

netsConv['f'].load_state_dict(torch.load(f"{models_conv_dir_path}Iter{9}_net_f_GFlash_Conv{modelConv_name}_{modelConv_version}_.pth", map_location=device), strict=False)
netsConv['b'].load_state_dict(torch.load(f"{models_conv_dir_path}Iter{9}_net_b_GFlash_Conv{modelConv_name}_{modelConv_version}_.pth", map_location=device), strict=False)

netsConv['f'].eval()
netsConv['b'].eval()

#---------------------
netsConv['f'] = netsConv['f'].to(device)#.cpu()
netsConv['b'] = netsConv['b'].to(device)#.cpu()
# modelConv_f = torch.nn.DataParallel(netsConv['f'])
# modelConv_b = torch.nn.DataParallel(netsConv['b'])

print(next(netsConv['f'].parameters()).is_cuda)
print(next(netsConv['b'].parameters()).is_cuda)

True
True
True
True


In [9]:
energy__shower_start = torch.rand(batch_size,1,device=device)
energy__shower_tot = torch.rand(batch_size,20,1,device=device)
energy__shower_out = torch.rand(batch_size,20,1,device=device)
steps_energy = torch.rand(batch_size,20,1,device=device)
energy_particle = torch.rand(batch_size,1,device=device)
x = torch.rand(batch_size,1,10,10,device=device)
x_tot = torch.rand(batch_size, 20, 1, 10, 10,device=device)
out = torch.rand(batch_size, 20, 1, 10, 10,device=device)
steps_voxel = torch.rand(batch_size,20,1,device=device)
y_current = torch.rand(batch_size,3,device=device)


In [10]:
forward_or_backward = 'f'
forward_or_backward_rev = 'b'

ts_tot_mean = []
ts_tot_std = []

n_reps = 10

for _ in range(n_reps):

    netsEnergy_ts = []
    netsConv_ts = []
    with torch.no_grad():
            for k in range(num_iter_energy):
                gamma = gammas_energy[k]

                t0 = time.time()
                #---------------------------------------------------------------------------------------------------#
                t_old = energy__shower_start + netsEnergy[forward_or_backward_rev](energy__shower_start, 
                                                                                steps_energy[:, k, :],
                                                                                energy_particle, energy__shower_start)
                if k == num_iter_energy-1:
                    energy__shower_start = t_old
                else:
                    z = torch.randn(energy__shower_start.shape, device=x.device)
                    energy__shower_start = t_old + torch.sqrt(2 * gamma) * z

                t_new = energy__shower_start + netsEnergy[forward_or_backward_rev](energy__shower_start, 
                                                                                steps_energy[:, k, :],
                                                                                energy_particle, energy__shower_start)
                energy__shower_tot[:, k, :] = energy__shower_start
                energy__shower_out[:, k, :] = (t_old - t_new)
                #---------------------------------------------------------------------------------------------------#
                netsEnergy_ts.append(time.time()-t0)


    with torch.no_grad():
        for k in range(num_iter_voxel):
            gamma = gammas_voxel[k]

            t0 = time.time()
            #---------------------------------------------------------------------------------------------------#
            t_old = x + netsConv[forward_or_backward_rev](x, steps_voxel[:, k, :], y_current, x)

            if k == num_iter_voxel-1:
                x = t_old
            else:
                z = torch.randn(x.shape, device=x.device)
                x = t_old + torch.sqrt(2 * gamma) * z

            t_new = x + netsConv[forward_or_backward_rev](x, steps_voxel[:, k, :], y_current,x )
            
            x_tot[:, k, :] = x
            out[:, k, :] = (t_old - t_new)
            #---------------------------------------------------------------------------------------------------#
            netsConv_ts.append(time.time()-t0)
    

    ts_tot = np.array(netsEnergy_ts) + np.array(netsConv_ts)
    ts_tot = ts_tot/batch_size*num_iter_voxel*1000

    print(ts_tot.mean())

    ts_tot_mean.append(ts_tot.mean())
    ts_tot_std.append(ts_tot.std())

/home/marcelomdu/miniforge3/envs/sb_ref/lib/python3.9/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


0.08270018100738526
0.12313406467437744
0.12322428226470947
0.12309808731079101
0.12334070205688477
0.12308645248413086
0.12358818054199219
0.12335894107818604
0.12344202995300294
0.12360556125640869


In [11]:
print(modelEnergy_name, modelConv_name)

SquirelsScoreNetwork SquirelsScoreNetworkConv


In [12]:
np.array(ts_tot_mean).mean()

0.11925784826278688

In [13]:
np.array(ts_tot_std).mean()

0.043249864224077214